In [68]:
import matplotlib.pyplot as plt
import polars as pl
import pandas as pd
import numpy as np
import json
import os
import sys

sys.path.append("..")

import seaborn as sns

from settings import (
    random_state,
    PROJECT_PATH,
    REGRESSION_TARGET,
    CLASSIFICATION_TARGET,
)
from catboost import CatBoostClassifier, Pool
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import recall_score, precision_score, f1_score
import mlflow
from mlflow import MlflowClient

In [3]:
def perform_cross_validation(
    X: pl.DataFrame,
    y: pl.Series,
    model,
    cross_val_type,
    scoring_metrics: tuple,
    groups=None,
):
    scores = cross_validate(
        model,
        X.to_numpy(),
        y.to_numpy(),
        cv=cross_val_type,
        return_train_score=True,
        return_estimator=True,
        scoring=scoring_metrics,
        groups=groups,
    )

    scores_dict = {}
    for metric in scoring_metrics:
        scores_dict["average_train_" + metric] = np.mean(scores["train_" + metric])
        scores_dict["train_" + metric + "_std"] = np.std(scores["train_" + metric])
        scores_dict["average_test_" + metric] = np.mean(scores["test_" + metric])
        scores_dict["test_" + metric + "_std"] = np.std(scores["test_" + metric])

    model.fit(X.to_numpy(), y.to_numpy())

    return scores, scores_dict, model

def get_features_most_importance(importances, feature_names, threshold=0.8):
    sorted_indices = np.argsort(importances)
    sorted_importances = importances[sorted_indices][::-1]
    sorted_feature_names = [feature_names[i] for i in sorted_indices][::-1]

    cumulated_importance = 0
    important_features = []

    for importance, feature_name in zip(sorted_importances, sorted_feature_names):
        cumulated_importance += importance
        important_features.append(feature_name)

        if cumulated_importance >= threshold:
            break

    return important_features

In [4]:
transactions = pl.read_parquet(
    os.path.join(PROJECT_PATH, "transactions_post_feature_engineering.parquet")
)


with open("../features_used.json", "r") as f:
    feature_names = json.load(f)

with open("../categorical_features_used.json", "r") as f:
    categorical_features = json.load(f)

numerical_features = [col for col in feature_names if col not in categorical_features]

In [90]:
transactions_v1 = transactions.filter(pl.col("annee_transaction") < 2020)

transactions_v2 = transactions.filter(
    pl.col("annee_transaction").is_between(2020, 2021)
)
features_1 = [
    "type_batiment_Appartement",
    "surface_habitable",
    "prix_m2_moyen_mois_precedent",
    "nb_transactions_mois_precedent",
    "taux_interet",
    "variation_taux_interet",
    "acceleration_taux_interet",
]

features_2 = features_1 + ["longitude", "latitude", "vefa"]

<b> Attention : </b> Assurez-vous de lancer ce code dans le même dossier où les modèles pré-covid ont été créés ! En effet, quand vous lancez votre MLflow UI (ainsi qu'un objet MlflowClient) sans arguments supplémentaires, le package va utiliser comme modèle registry ceux du dossier actuel. Plus précisement, Mlflow va créer un dossier mlruns, où seront stockés tous vos modèles. 

Il suffit alors de : 
* lancer dans votre Terminal la commande mlflow ui en précisant l'adresse du dossier mlruns comme ceci : mlflow ui --backend-store-uri adresse_de_votre_choix
* Réaliser la même opération côté Python avec la commande mlflow.set_tracking_uri("adresse_de_votre_choix")